In [3]:
%run Imports\&functions.ipynb

In [6]:
# list of binetflow files
#READ ME: This is only a fraction of the entire dataset
#Github does not allow file size above 100mb
#This is only 78mb of an 2.73 Gb total
binetflow_list =[ "binetflow/capture20110816-2.binetflow",
                 "binetflow/capture20110818-2.binetflow",
                 "binetflow/capture20110819.binetflow"]

# read binetflows into df1-df13 variables
i = 1
for binetflow in binetflow_list:
    globals()["df" + str(i)] = pd.read_csv(binetflow)
    i+=1

#place newly read pandas into list for easier data processing
# df_list = [df1,df2,df3,df4,df5,
#            df6,df7,df8,df9,df10,
#            df11,df12,df13]

#changed for smaller dataset
df_list = [df1,df2,df3]

In [ ]:
#Drop null values
#Most from one column but Dropping column lost more information than dropping nulls
for df in df_list:
    df.dropna(inplace=True)

In [ ]:
#Change Label column from string to boolean for modeling
for df in df_list:
    df.Label = df.Label.str.contains("Botnet")

In [ ]:
for df in df_list:
    df.drop('StartTime',axis=1, inplace=True)
    df.drop('SrcAddr',axis=1, inplace=True)
    

In [ ]:
#adding dataframes together for use in cloud instance
total_df = df1.append(df2)
total_df = total_df.append(df3)
total_df = total_df.append(df4)
total_df = total_df.append(df5)
total_df = total_df.append(df6)
total_df = total_df.append(df7)
total_df = total_df.append(df8)
total_df = total_df.append(df9)
total_df = total_df.append(df10)
total_df = total_df.append(df11)
total_df = total_df.append(df12)
total_df = total_df.append(df13)
#Split up append functions individually rather than loop
#Because of checking shape and errors throughout 

In [ ]:
#Noticed identical rows when all dataframes added together
total_df.drop_duplicates(inplace=True)

In [ ]:
#regex pattern for grabbing first 6 digits of IP address
pattern ="^\d{1,3}\.\d{1,3}\."
#replace column values with pattern grab
total_df['DstAddr'] = total_df['DstAddr'].map(lambda x: re.findall(pattern,x)[0])

In [ ]:
#get dummies of categorical values that don't have value counts over 30
total_df = pd.get_dummies(total_df, columns=['State','Proto','Dir'], drop_first=True)

In [ ]:
total_df.shape

In [ ]:
#Random sampling of minority and majority values
#READ THIS: n_samples value significantly lowered for notebooks to be able to run on laptops 
#Label value is what we are trying to predict

df_majority = total_df[total_df.Label==False]
df_minority = total_df[total_df.Label==True]

df_majority_downsampled = resample(df_majority, 
                                 replace=False,     # sample with replacement
                                 n_samples=50000,# to match majority class
                                 random_state=42) # reproducible results

# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=False,     # sample with replacement
                                 n_samples=50000,# to match minority class
                                 random_state=42) # reproducible results
 

In [ ]:
df_minority_upsampled[df_minority_upsampled.Label==True].shape

In [ ]:
df_majority_downsampled[df_majority_downsampled.Label==False].shape

In [ ]:
# Combine upsampled minority class with majority class
sampled_df = pd.concat([df_minority_upsampled, df_majority_downsampled])

In [ ]:
sampled_df[sampled_df.Label==True].shape

In [ ]:
#Grab top 100 value counts of the three remaining categorical columns

top_dstaddr = sampled_df.DstAddr.value_counts().index[:100]
top_dport = sampled_df.Dport.value_counts().index[:100]
top_sport = sampled_df.Sport.value_counts().index[:100]

# func that returns a dummified DataFrame of significant dummies in a given column
def dum_sign(dummy_col, threshold=1):

    # removes the bind
    dummy_col = dummy_col.copy()

    # what is the ratio of a dummy in whole column
    count = pd.value_counts(dummy_col) / len(dummy_col)

    # cond whether the ratios is higher than the threshold
    mask = dummy_col.isin(count[count > threshold].index)

    # replace the ones which ratio is lower than the threshold by a special name
    #dummy_col[~mask] = "others"

    return pd.get_dummies(dummy_col, prefix=dummy_col.name)

# assign values to pandas dataframe
dstaddr_dummies= dum_sign(top_dstaddr)
dport_dummies= dum_sign(top_dport)
sport_dummies= dum_sign(top_sport)

In [ ]:
sampled_df= sampled_df.join(dstaddr_dummies)

In [ ]:
sampled_df= sampled_df.join(dport_dummies)

In [ ]:
# sampled_df= sampled_df.join(sport_dummies)

In [ ]:
# sampled_df = pd.get_dummies(df, columns=['DstAddr'], drop_first=True)
# sampled_df.isnull().sum()
sampled_df[sampled_df.Label==True].shape

In [ ]:
#remove dummied columns to save memory
sampled_df = sampled_df.drop(axis=1,labels=['DstAddr','Dport','Sport'])

In [ ]:
sampled_df= sampled_df.fillna(0)

In [ ]:
from sklearn.model_selection import train_test_split

y = sampled_df['Label']
X = sampled_df.drop(axis=1,columns=['Label'])

# Import and do Train/Test Split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25,random_state = 42)

In [ ]:
from sklearn.linear_model import LogisticRegression

logmod = LogisticRegression(max_iter=1000,verbose=1)

# Fit on training data.
logmod.fit(X_train, y_train)

# Print coefficients.
print('Intercept:', logmod.intercept_)
print('Coef(s):', logmod.coef_)

logmod.score(X_test,y_test)

In [ ]:
# Import libraries and modules

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

In [ ]:
model = Sequential()

n_input = X_train.shape[1]
n_hidden = n_input

model.add(Dense(n_hidden, input_dim=n_input, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,
          y_train,
          batch_size = 589,
          epochs = 1000,
          verbose = 1)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
labels = model.metrics_names
print(f'{labels[0]}: {score[0]}')
print(f'{labels[1]}: {score[1]}')

In [ ]:
model.summary()
